## Closed Loop Simulation Notebook

This notebook breaks down the closed loop simulation in different jupyter notebook cells. The aim of this notebook is to ease the debugging and development of the different packages involved in the Optical Feedback Control loop.

### Required python packages

In [1]:
import argparse
import logging
import os
import sys

from lsst.ts.phosim.CloseLoopTask import CloseLoopTask

In [2]:
from tqdm import tqdm

### Default arguments and initializations

Initialize default arguments to run the closed loop and initialize the ClosedLoopTask

In [3]:
# Set the parser
parser = argparse.ArgumentParser(
    description="Run AOS closed-loop simulation (default is amplifier files)."
)
parser = CloseLoopTask.setDefaultParser(parser)
parser = CloseLoopTask.setImgParser(parser)

# Get the default arguments
sys.argv = ['-f']
args = parser.parse_args()

# Print default arguments
print(args)

logger = logging.getLogger()
logger.setLevel(args.log_level)

# Initialize the ClosedLoopTask
closeLoopTask = CloseLoopTask()

Namespace(boresightDeg=[0, 0], clobber=False, eimage=False, filterType='', inst='comcam', iterNum=5, log_level=20, m1m3FErr=0.05, numOfProc=1, output='', rotCam=0.0, skyFile='')


### Set paths and arguments

Customize the following arguments to run the simulation you are interested in. If not stated in the following code cell, the arguments are taken to be the default ones. Note that this paths should be changed if another user is running them and wants to change the directory addresses.

In [4]:
PHOSIMPATH = "/project/gmegias/aos/lsst_stack/phosim_syseng4/"
AOCLCOUTPUTPATH = "/project/gmegias/aos/lsst_stack/ts_phosim/output/"
os.environ["PHOSIMPATH"] = PHOSIMPATH
os.environ["AOCLCOUTPUTPATH"] = AOCLCOUTPUTPATH

In [18]:
args.inst = 'comcam' 
args.numOfProc = 32 
args.boresightDeg = [0.03, -0.02]
args.skyFile = '/project/gmegias/aos/lsst_stack/ts_phosim/tests/testData/sky/skyComCam.txt'
args.output = '/project/gmegias/aos/perturbations/imgCloseLoop_test7/'

### Intializations and initial configurations

First, initialize the variables from the arguments dictionary.

In [19]:
boresight = args.boresightDeg
rotCamInDeg = args.rotCam
useEimg = args.eimage
m1m3ForceError = args.m1m3FErr
numPro = args.numOfProc
iterNum = args.iterNum
doErsDirCont = args.clobber
pathSkyFile = args.skyFile

Check and set the required configurations to run the simulation loop.

In [20]:
# Check the input arguments
camType, instName = closeLoopTask.getCamTypeAndInstName(args.inst)
filterType = closeLoopTask.getFilterType(args.filterType)
baseOutputDir = closeLoopTask.checkAndCreateBaseOutputDir(args.output)

if doErsDirCont:
    closeLoopTask.eraseDirectoryContent(baseOutputDir)

closeLoopTask.checkBoresight(boresight, args.skyFile)

closeLoopTask.assignImgType(useEimg)

# Configure the components
closeLoopTask.configSkySim(instName, pathSkyFile = args.skyFile, starMag=15)

pathIsrDir = closeLoopTask.createIsrDir(baseOutputDir)
closeLoopTask.configWepCalc(
    camType, pathIsrDir, filterType, boresight, rotCamInDeg, useEimg=useEimg
)

closeLoopTask.configOfcCalc(instName)
closeLoopTask.configPhosimCmpt(
    filterType, rotCamInDeg, m1m3ForceError, numPro
)

# Set the defocal distance for WEP calculator based on the setting
# file in the telescope
closeLoopTask.setWepCalcWithDefocalDist()

### Generate butler gen3 repository.

In [21]:
# generate bluter gen3 repo if needed
butlerRootPath = os.path.join(baseOutputDir, "phosimData")
if closeLoopTask.useCcdImg():
    closeLoopTask.generateButler(butlerRootPath, instName)
    closeLoopTask.generateRefCatalog(
        instName=instName,
        butlerRootPath=butlerRootPath,
        pathSkyFile=pathSkyFile,
    )
    
closeLoopTask.phosimCmpt.tele.setInstName(camType)

### Simulation loop initialization

Initialize the state of the telescope to run the simulation. This cell also sets common file and directory names required for the closed loop simulation.

In [22]:
# Set the telescope state to be the same as the OFC
state0 = closeLoopTask.ofcCalc.ofc_controller.aggregated_state
closeLoopTask.phosimCmpt.setDofInUm(state0)

# Get the list of referenced sensor name (field positions)
refSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
refSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)

# Common file and directory names
opdZkFileName = "opd.zer"
opdPssnFileName = "PSSN.txt"
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
dofInUmFileName = "dofPertInNextIter.mat"
fwhmItersFileName = "fwhmIters.png"

# Specific file names to the amplifier/eimage
wfsZkFileName = "wfs.zer"

### Run simulation for each iteration

In [ ]:
# Do the iteration
obsId = 9006000

for iterCount in tqdm(range(iterNum)):
    print(iterCount)

    # Set the observation Id
    closeLoopTask.phosimCmpt.setSurveyParam(obsId=obsId)

    # The iteration directory
    iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

    # Set the output directory
    outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
    closeLoopTask.phosimCmpt.setOutputDir(outputDir)

    # Set the output image directory
    outputImgDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)
    closeLoopTask.phosimCmpt.setOutputImgDir(outputImgDir)

    # Generate the OPD image
    print("Preparing PhoSim....")
    argString = closeLoopTask.phosimCmpt.getOpdArgsAndFilesForPhoSim(instName)

    print("PhoSim prepared....")
    closeLoopTask.phosimCmpt.runPhoSim(argString)

    # Analyze the OPD data
    # Rotate OPD in the reversed direction of camera
    print("Analyzing OPD data....")
    closeLoopTask.phosimCmpt.analyzeOpdData(
        instName,
        zkFileName=opdZkFileName,
        rotOpdInDeg=-rotCamInDeg,
        pssnFileName=opdPssnFileName,
    )
    print("OPD data analyzed....")
    
    # Get the PSSN from file
    pssn = closeLoopTask.phosimCmpt.getOpdPssnFromFile(opdPssnFileName)
    print("Calculated PSSN is %s." % pssn)

    # Get the GQ effective FWHM from file
    gqEffFwhm = closeLoopTask.phosimCmpt.getOpdGqEffFwhmFromFile(opdPssnFileName)
    print("GQ effective FWHM is %.4f." % gqEffFwhm)

    # Set the FWHM data
    print("Setting FWHM data...")
    fwhm, sensor_id = closeLoopTask.phosimCmpt.getListOfFwhmSensorData(
        opdPssnFileName, refSensorIdList
    )

    closeLoopTask.ofcCalc.set_fwhm_data(fwhm, sensor_id)
    print("FWHM data set.")

    # Generate the sky images and calculate the wavefront error
    if closeLoopTask.useCcdImg():
        listOfWfErr = closeLoopTask._calcWfErrFromImg(
            obsId, butlerRootPath = butlerRootPath, instName=instName, snap=0
        )
    else:
        # Simulate to get the wavefront sensor data from WEP
        listOfWfErr = closeLoopTask.phosimCmpt.mapOpdDataToListOfWfErr(
            opdZkFileName, refSensorIdList
        )

    # Record the wavefront error with the same order as OPD for the
    # comparison
    if closeLoopTask.useCcdImg():
        closeLoopTask.phosimCmpt.reorderAndSaveWfErrFile(
            listOfWfErr,
            refSensorNameList,
            getCamera(instName),
            zkFileName=wfsZkFileName,
        )

    # Calculate the DOF
    wfe = np.array(
        [sensor_wfe.getAnnularZernikePoly() for sensor_wfe in listOfWfErr]
    )
    sensor_ids = np.array(
        [sensor_wfe.getSensorId() for sensor_wfe in listOfWfErr]
    )

    closeLoopTask.ofcCalc.calculate_corrections(
        wfe=wfe,
        field_idx=sensor_ids,
        filter_name=str(filterType),
        gain=-1,
        rot=rotCamInDeg,
    )

    # Set the new aggregated DOF to phosimCmpt
    dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state
    closeLoopTask.phosimCmpt.setDofInUm(dofInUm)

    # Save the DOF file
    closeLoopTask.phosimCmpt.saveDofInUmFileForNextIter(
        dofInUm, dofInUmFileName=dofInUmFileName
    )

    # Add the observation ID by 10 for the next iteration
    obsId += 10

### Summarize the FWHM and plot pssn profiles

In [ ]:
# Summarize the FWHM
pssnFiles = [
    os.path.join(
        baseOutputDir,
        "%s%d" % (iterDefaultDirName, num),
        outputImgDirName,
        opdPssnFileName,
    )
    for num in range(iterNum)
]
saveToFilePath = os.path.join(baseOutputDir, fwhmItersFileName)
plotFwhmOfIters(pssnFiles, saveToFilePath=saveToFilePath)